<a href="https://colab.research.google.com/github/virajpai/eda_report/blob/dev/EdaReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
#from common.eda_utils.eda_format import EDA Formatter

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

# Excel Formatting
from sys import maxsize

from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.formatting.rule import ColorScaleRule

In [119]:
class EDA_Formatter:
  def __init__(self, path='..\\reports\\EDA raw. xlsx', model_type='Target', conditional_color='red '):

    self.input_path = path
    self.output_path = 'formatted.xlsx'.join(path. split('.xlsx'))
    self.type = model_type
    self.color = conditional_color
    self.r =1
    self.c = 1

    self.setup_workbook()
    self.run_formatter()

  def setup_workbook(self):
    self.wb = Workbook()
    ws2 = load_workbook(self.input_path) ['ROC Report']
    ws2._parent = self.wb
    self.wb._add_sheet(ws2)
    self.ws = self.wb.worksheets[0]
    self.ws.title = 'Detailed EDA'
    self.wb.move_sheet('Detailed EDA', offset=1)
    self.set_column_widths()

  def run_formatter(self):
    df = pd.read_excel(self.input_path, 'Detailed EDA', engine='openpyxl')
    df.rename(columns={'value': 'Value', 'count': 'Frequency', 'sum': self.type, 'mean': self.type + ' Rate'}, inplace=True)
    df.insert(loc=3, column='Freq Distribution', value=0)
    df.insert(loc=6, column=r'% of Total ' + self. type, value=0)
    df.insert(loc=7, column='Lift', value=0)

    df2 = pd. DataFrame (columns=df. columns)
    init_val = df.loc[0][0]

    for index in df.index:
      if df.loc[index][0] != init_val:
        df2.reset_index(inplace=True, drop=True)
        self.write_to_excel(df2)
        self.r += len(df2.index) + 3

        df2 = pd. DataFrame(columns=df.columns)
        init_val = df.loc[index][0]

      df2.loc[index] = df.loc[index]

    df2.reset_index(inplace=True, drop=True)
    self.write_to_excel(df2)
    self.wb.save(self.output_path)

  @staticmethod
  def is_number (n) :
    try:
      float(n)
    except ValueError:
      return False
    return True

  @staticmethod
  def sort(df):
    df['Value'] = df['Value'].apply(pd.to_numeric)
    df.sort_values(by='Value', inplace=True)
    df = df.reset_index(drop=True)

    return df

  def set_column_widths(self):
    width_list = [43, 60, 10, 15, 10, 14, 19, 11]
    for index, width in enumerate(width_list):
      char = get_column_letter(index + self.c)
      self.ws.column_dimensions[char].width = (width+0.78)

  def number_format(self, df_rows):
    # print(df_rows)
    char = get_column_letter(self.c+1)

    prev_cell = self.ws[char+str(self.r)]
    prev_val = prev_cell.value
    #print(prev_val)
    for row in range (1, df_rows + 1):
      cell = self.ws[char + str(self.r + row)]
      val = cell.value

      if (row == 1):
        cell.value = '<= ' + str(val)
      elif (row == df_rows):
        cell.value = '> ' + str(prev_val)
      else:
        cell.value = '> ' + str(prev_val) + ' & <= ' + str(val)

      prev_val = val

  def df_formatter(self, rows, cols):
    thin_border = Border(left=Side(border_style='thin'),
                          right=Side(border_style='thin'),
                          top=Side(border_style='thin'),
                          bottom=Side(border_style='thin'))
    font = Font(name='Calibri',
                size=11,
                bold=True)

    fill = PatternFill(fill_type='solid',
                        start_color='E4DFEC',
                        end_color='E4DFEC')

    center_alignment = Alignment(
        horizontal='center', vertical='center')
    left_alignment = Alignment(
        horizontal='left', vertical='center')
    right_alignment = Alignment(
        horizontal= 'right', vertical='center')

    for col in range(0, cols):
      char = get_column_letter(col + self.c)
      header_cell = self.ws[char + str(self.r)]

      header_cell.border = thin_border
      header_cell.font = font
      header_cell.alignment = center_alignment
      header_cell.fill = fill

      for row in range(0, rows):
        data_cell = self.ws[char + str(self.r+row+1)]
        if col == 0:
          data_cell.alignment = center_alignment
        else:
          if(col == 3 or col == 5 or col == 6):
            data_cell.number_format = '0.00%'
          elif(col == 7):
            data_cell.number_format ='0.00'
          if col == 1:
            data_cell.alignment = left_alignment
          else:
            data_cell. alignment = right_alignment

    char = get_column_letter(self. c)
    self.ws.merge_cells(char+str(self.r+1) + ':' + char + str(self.r+rows))

  def cond_format(self, start_row, end_row, start_col, end_col):

      if(self.color == 'red' or self.color == 'Red'):
        High_value = 'F8696B'
      else:
        High_value = '63BE7B'

      for col in range(start_col, end_col+1):
        char = get_column_letter(col)
        if (self.color == 'color' or self.color == 'Color'):
          self.ws.conditional_formatting.add(char+str(start_row) + ':' + char + str(end_row),
                                             ColorScaleRule(start_type='percentile', start_value=0, start_color='F8696B',
                                             mid_type='percentile', mid_value=50, mid_color='FFEB84',
                                             end_type='percentile', end_value=100, end_color= '63BE7B'))
        else:
          self.ws.conditional_formatting.add(char+str(start_row)+':'+char+str(end_row),
                                               ColorScaleRule(start_type='percentile', start_value=0, start_color='FCFCFF',
                                               end_type='percentile', end_value=100, end_color=High_value))

  def write_to_excel(self, df):
    num_flag = False

    # Writing header
    for col in range (0, len(df.columns)):
      char = get_column_letter(self.c + col)
      cell = self. ws[char + str(self.r)]
      cell.value = df.columns[col]

    # Sorting if numeric after replacing inf with maxsize
    if (self.is_number(df.loc[0][1]) or df.loc[0][1] == 'inf'):
      for index in df.index:
        if df.loc[index][1] == 'inf':
          # df. loc[index] [1] = maxsize
          df.at[index, 'Value'] = maxsize
      df = self.sort(df)
      df['Value'] = df['Value'].round(decimals=2)
      num_flag = True
      #display(df)

    # Writing values
    for row in range(0, len(df.index)):
      for col in range (0, len(df.columns)):
        char = get_column_letter(self.c+col)
        cell = self.ws[char+str(self.r+row+1)]
        if col == 3:
          char2 = get_column_letter(self.c+2)
          cell.value = '=' + char2 + str(self.r+row+1) + "/SUM(" +char2+ '$' + str(
              self.r+1) + ':' + char2 + '$' + str(self.r+len(df.index))+ ")"
        elif col == 6:
          char2 = get_column_letter(self.c+4)
          cell.value = '=' + char2 + str(self.r+row+1) + "/SUM(" +char2+ '$' + str(
              self.r+1) + ':' + char2 + '$' + str(self.r+len(df.index))+ ")"
        elif col == 7:
          char2 = get_column_letter(self.c+3)
          char3 = get_column_letter(self.c+6)
          cell.value = '=' + char3 + \
            str(self.r+row+1)+'/'+char2+str(self.r+row+1)
        else:
          cell.value = df.loc[row][col]

        #print(cell.value)

    if num_flag:
      #print(num_flag)
      self.number_format(len(df.index))

    self.df_formatter(rows=len(df.index), cols=len(df.columns))
    self.cond_format(start_row=self.r+1,
                     end_row=self.r+len(df.index), start_col=self.c+5, end_col=self.c+7)









In [105]:
class EDAReport():
  def __init__(self, data: pd.DataFrame, target: str, report_path: str,
               ignore_cols: list[str] = None, cat_label_enco_thresh: float = 0.05,
               num_min_samples_leaf: float = 0.1, conditional_color: str = 'red') -> None:

    grp_data = self.__get_full_eda(data, target, ignore_cols=ignore_cols,
                                   cat_label_enco_thresh=cat_label_enco_thresh,
                                   num_min_samples_leaf=num_min_samples_leaf)
    roc_data = self.__get_roc_auc(data, target, ignore_cols=ignore_cols,
                                  cat_label_enco_thresh=cat_label_enco_thresh,
                                  num_min_samples_leaf=num_min_samples_leaf)

    # display(roc_data)

    xw = pd.ExcelWriter(report_path, engine="openpyxl")
    roc_data.to_excel(xw, sheet_name='ROC Report', index=False)
    grp_data.to_excel(xw, sheet_name='Detailed EDA', index=False)
    xw.close()

    EDA_Formatter(path=report_path, model_type=target, conditional_color=conditional_color)

  def __get_full_eda(self, data: pd.DataFrame, target: str,
                   ignore_cols: list[str] = None, cat_label_enco_thresh: float = 0.05,
                   num_min_samples_leaf: float = 0.1) -> pd.DataFrame:

      # Get all required columns
      cols = data.columns.tolist()
      cols.remove(target)

      if ignore_cols is not None:
        cols = [col for col in cols if col not in ignore_cols]

      y = data[target].copy()

      # EDA
      all_grp_dfs = []
      for col in cols:
        # Numeric
        isnum = str(data[col].dtype).startswith(
            'float') | str(data[col].dtype).startswith('int')

        if isnum:
          # print(col, ': ', isnum)
          # print(f'num_min_sample_leaf:{num_min_samples_leaf}')
          X = np.array(data[col].fillna(data[col].median()
                                        ).values.tolist()).reshape(-1, 1)

          dt = DecisionTreeClassifier(
              class_weight='balanced', min_samples_leaf=num_min_samples_leaf)

          dt.fit(X, y)

          thresholds = np.sort(np.unique(dt.tree_.threshold))
          thresholds = np.append(thresholds, np.inf)
          X = np.digitize(X, thresholds, right=True)

          x_map = dict(enumerate(map(str, thresholds)))

          # print(x_map)

          grp_df = pd.DataFrame(X, columns=[col])
          grp_df[col].replace(x_map, inplace=True)
          grp_df[target] = y

          grp_df = grp_df.groupby(col)[target].agg(
              ['count', 'sum', 'mean']).reset_index()
          grp_df.insert(0, 'Column ', col)
          grp_df.rename(columns={col: 'value'}, inplace=True)

          # display(grp_df)

        else:
          grp_df = data.groupby(col)[target].agg(
              ['count', 'sum', 'mean']).reset_index()
          grp_df.insert(0, 'Column ', col)
          grp_df.rename(columns={col: 'value'}, inplace=True)


        all_grp_dfs.append(grp_df)

      return pd.concat(all_grp_dfs)

  def __get_roc_auc(self, data: pd.DataFrame, target: str,
               ignore_cols: list[str] = None, cat_label_enco_thresh: float = 0.05,
               num_min_samples_leaf: float = 0.1) -> pd.DataFrame:
    #print('roc')
    # Get all required columns
    cols = data.columns.tolist()
    cols.remove(target)

    #print(ignore_cols)
    #print(cols)

    if ignore_cols is not None:
      cols = [col for col in cols if col not in ignore_cols]


    y = data[target].copy()
    #print(cols)
    # EDA
    all_auc_dfs = []
    for col in cols:
      #print(col)
      # Numeric
      isnum = str(data[col].dtype).startswith(
          'float') | str(data[col].dtype).startswith('int')

      if isnum:
        X = np.array(data[col].fillna(data[col].median()
                                      ).values.tolist()).reshape(-1, 1)
        dt = DecisionTreeClassifier(class_weight='balanced',
                                    min_samples_leaf = num_min_samples_leaf)
      else:
        X = np.array(data[col].fillna(data[col].mode()[0]
                                      ).values.tolist()).reshape(-1, 1)

        label_enc = {}
        i = 0
        for unx in np.unique(X):
          rate = np.where(X == unx, 1, 0).sum() / len(X)

          if rate >= cat_label_enco_thresh:
            i = i + 1
            label_enc.update({unx: i})
          else:
            label_enc.update({unx: 0})
        #print(label_enc)
        map_label_enc = np.vectorize(label_enc.get)
        #print(col)
        #print(map_label_enc)
        X = map_label_enc(X)
        dt = DecisionTreeClassifier(class_weight='balanced')

      # KFold Decision Tree Classifier
      kf = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
      results = []
      i = 0
      for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        i = i + 1
        dt.fit(X_train, y_train)
        yproba = dt.predict_proba(X_test)
        auc = metrics.roc_auc_score(y_test, yproba[:, 1])
        results.append(auc)

      all_auc_dfs.append({
          'Column': col,
          'ROC AUC': np.median(results)
      })

      # print(all_auc_dfs)
    return pd.DataFrame(all_auc_dfs)

In [106]:
from sklearn.datasets import load_wine, load_breast_cancer
import random

In [107]:
wine_dataset = load_wine(as_frame=True)
bc_dataset = load_breast_cancer(return_X_y=True, as_frame=True)

In [108]:
df = bc_dataset[0]

In [109]:
# df = wine_dataset.data

In [110]:
# wine_dataset.target_names

In [111]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [112]:
rnd_set = ['a', 'b', 'c']
df['Random Cat'] = random.choices(rnd_set, k=len(df))

In [113]:
df['Target'] = bc_dataset[1]

In [114]:
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Random Cat,Target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,a,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,c,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,c,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,b,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,b,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,c,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,c,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,b,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,a,0


In [115]:
df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Target
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


In [116]:
df[['mean radius', 'mean texture', 'Random Cat', 'Target']]

,mean radius,mean texture,Random Cat,Target
0,17.99,10.38,a,0
1,20.57,17.77,c,0
2,19.69,21.25,c,0
3,11.42,20.38,b,0
4,20.29,14.34,b,0
...,...,...,...,...
564,21.56,22.39,c,0
565,20.13,28.25,c,0
566,16.60,28.08,b,0
567,20.60,29.33,a,0


In [120]:
EDAReport(df, 'Target', './report.xlsx', num_min_samples_leaf=0.1)